Great! Below is a step-by-step LangGraph Advanced Agent tutorial for Jupyter Notebook, with all code blocks broken down into clear, logical parts.

# Advanced Research Agent with FastAPI, LangGraph, and Streaming

This notebook guides you through building an advanced AI agent that:

- Expands queries
- Searches the web (via Tavily)
- Summarizes results using LLM
- Streams responses in real-time using FastAPI

We'll build this using:

- FastAPI
- LangGraph
- LangChain
- OpenAI
- Tavily

### 1. Install Required Packages

In [ ]:
!pip install fastapi uvicorn nest_asyncio langchain-openai openai sse-starlette pydantic tavily

nest_asyncio is needed to run uvicorn within Jupyter.

In [2]:
import nest_asyncio
nest_asyncio.apply()


### 2. Set API Keys
You can use environment variables or set them inline for this notebook.

In [ ]:
import os

# Set your OpenAI key (replace with your key)
os.environ["OPENAI_API_KEY"] = "Your_API_Key"  # Replace with your actual key
os.environ["TAVILY_API_KEY"] = "Your_Tavily_API_Key"  # Replace with your actual key


### 3. Define Models and Agent State

In [4]:
from typing import TypedDict, List
from pydantic.v1 import BaseModel, Field

# Tavily search result format
class SearchResult(BaseModel):
    url: str = Field(..., description="The URL of the search result")
    title: str = Field(..., description="The title of the search result")
    raw_content: str = Field(..., description="The raw content of the search result")

# Agent state passed between nodes
class ResearchState(TypedDict):
    query: str                      # The original user query
    expanded_query: str             # Expanded version of the query
    documents: List[SearchResult]   # Results returned from the search tool
    summary: str                    # Summary of the documents


### 4. Set Up LangGraph Nodes (Agent Steps)

In [ ]:
from langchain_openai import ChatOpenAI
from tavily import TavilyClient
from langgraph.config import get_stream_writer


# ---- LLM SETUP ----
llm = ChatOpenAI(
    model_name="openai/gpt-4.1-mini",
    streaming=True,
)


# ---- Nodes ----
# Node 1: Expand query
async def expand_query(state: ResearchState):
    response = await llm.ainvoke(
        f"You are a query writer agent. Rewrite this query to be more specific:\n\n'{state['query']}'.\n\n Only return the rewritten query without any additional text."
    )
    return {"expanded_query": response.content.strip()}

# Node 2: Run search tool
def run_tool_search(state: ResearchState):
    tavily_client = TavilyClient()
    expanded_query = state["expanded_query"]
    print(f"\n Searching for: {expanded_query}")
    #  Write to stream manually
    writer = get_stream_writer()
    writer(
        {
            "node": "run_tool_search",
            "token": "\nSearching academic sources and research databases...\n",
        }
    )

    try:
        response = tavily_client.search(
            query=expanded_query,
            max_results=3,
            include_raw_content=True,
        )
    except Exception as e:
        print(f" Error during search: {e}")
        return {"documents": []}

    documents = []
    for result in response.get("results", []):
        if result.get("raw_content") and result.get("url") and result.get("title"):
            documents.append(
                SearchResult(
                    url=result["url"],
                    title=result["title"],
                    raw_content=result["raw_content"],  # Optional truncation
                )
            )

    print(f"Found {len(documents)} results")
    print("Results:")
    for i, doc in enumerate(documents, 1):
        print(f"\nResult {i}:")
        print(f"Title: {doc.title}")
        print(f"URL: {doc.url}")
        print(f"Content Preview:\n{doc.raw_content}...\n")
    return {"documents": documents}

# Node 3: Summarize documents
async def summarize_documents(state: ResearchState):
    summaries = []
    for doc in state["documents"]:
        summaries.append(
            f"Title: {doc.title}\nURL: {doc.url}\nContent: {doc.raw_content}\n"
        )

    prompt = (
        "Please summarize the following search results in a clear and concise manner.\n\n"
        + "\n\n".join(summaries)
        + f"\n\nAt the beginning of your summary, include this sentence:\n"
        f"\"This is a summary of the results for '{state['query']}'.\"\n"
    )
    response = await llm.ainvoke(prompt)
    return {"summary": response.content.strip()}


### 5. Build LangGraph Workflow

In [6]:
from langgraph.graph import StateGraph, START, END

graph_builder = StateGraph(ResearchState)
graph_builder.add_node("expand_query", expand_query)
graph_builder.add_node("run_tool_search", run_tool_search)
graph_builder.add_node("summarize_documents", summarize_documents)

graph_builder.add_edge(START, "expand_query")
graph_builder.add_edge("expand_query", "run_tool_search")
graph_builder.add_edge("run_tool_search", "summarize_documents")
graph_builder.add_edge("summarize_documents", END)

graph = graph_builder.compile()

### 6. Create FastAPI Streaming Endpoint

In [ ]:
from fastapi import FastAPI, Request
from fastapi.responses import StreamingResponse
import json

app = FastAPI(title="LangGraph Research Assistant")

@app.post("/research/stream")
async def stream_graph_output(request: Request):
    print("Received request")
    data = await request.json()
    query = data.get("query", "")

    async def stream():
        expand_query_bool = False
        async for mode, chunk in graph.astream(
            {"query": query}, stream_mode=["messages", "custom"]
        ):
            if mode == "messages":
                node = chunk[1].get("langgraph_node")
                if node == "expand_query" and not expand_query_bool:
                    expand_query_bool = True
                    payload = {
                        "node": node,
                        "token": "\nBroadening User query for better results...",
                    }
                    yield f"data: {json.dumps(payload)}\n\n"
                elif node == "summarize_documents" and chunk[0].content:
                    if chunk[0].content and node == "summarize_documents":
                        payload = {"node": node, "token": chunk[0].content}
                        yield f"data: {json.dumps(payload)}\n\n"

            elif mode == "custom":
                # This is where the run_tool_search message comes in
                token = chunk.get("token", "")
                node = chunk.get("node", "")
                yield f"data: {json.dumps({'node': node, 'token': token})}\n\n"

    return StreamingResponse(stream(), media_type="text/event-stream")

### 7. Run FastAPI Server Inside Notebook

In [ ]:
import nest_asyncio
import uvicorn
import threading

nest_asyncio.apply()

def run_server():
    uvicorn.run(app, host="127.0.0.1", port=8000)

# Only run server if not already running
if "server_thread" not in globals():
    server_thread = threading.Thread(target=run_server, daemon=True)
    server_thread.start()
    print(" FastAPI server started on http://127.0.0.1:8000")
else:
    print(" Server is already running.")


 FastAPI server started on http://127.0.0.1:8000


INFO:     Started server process [47868]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Received request
INFO:     127.0.0.1:63042 - "POST /research/stream HTTP/1.1" 200 OK

 Searching for: Impact of artificial intelligence applications on personalized learning outcomes in K-12 education
Found 3 results
Results:

Result 1:
Title: Artificial Intelligence for Personalized Learning in K-12 Education. A ...
URL: https://link.springer.com/chapter/10.1007/978-3-031-67351-1_25
Content Preview:
Advertisement

Artificial Intelligence for Personalized Learning in K-12 Education. A Scoping Review

Part of the book series:
Communications in Computer and Information Science ((CCIS,volume 2076))

Included in the following conference series:

605 Accesses

1 
Citations

Abstract

The ongoing evolution of Artificial Intelligence technology is able to influence different fields of human experience, also including communication and professional activities. Artificial Intelligence has been playing a relevant role in education and literature has already addressed the design, impact, and chal

### 8. Test Endpoint from Notebook

We can test our **"/research/stream"** endpoint using requests.

In [9]:
import requests
import json

res = requests.post(
    "http://localhost:8000/research/stream",
    json={"query": "AI and education"},
    stream=True,
)

for line in res.iter_lines():
    if line and line.startswith(b"data: "):
        data = json.loads(line[6:])
        try:
            print(
                f"{data.get('token')}",
                end="",
                flush=True,
            )
        except json.JSONDecodeError:
            print("\n Warning: Could not decode server response.")


Broadening your query for better results...
Searching academic sources and research databases...
This is a summary of the results for "AI and education."

1. **Artificial Intelligence for Personalized Learning in K-12 Education (Springer, 2024)**  
This scoping review highlights the significant positive impact of AI technologies on personalized learning in K-12 settings. AI-enabled tools, particularly Intelligent Tutoring Systems (ITS) and adaptive learning platforms, are predominantly used for STEM subjects and language learning. These systems provide tailored instruction, instant feedback, and dynamically adjust to individual student needs, improving learning outcomes and retention. AI also aids in early identification of at-risk students through predictive analytics and supports inclusive education by facilitating accessibility for students with disabilities. However, challenges include concerns about data privacy, algorithmic bias, over-reliance on automated systems, and limited p

### Summary

We just built a powerful, real-time streaming research assistant using:
- LangGraph to orchestrate multi-step logic
- LangChain + OpenAI for query expansion and summarization
- Tavily for live search results
- FastAPI to serve everything with streaming
- We can now plug this into frontends like Streamlit, React, or use it in pipelines or Slackbots.